In [1]:
import pandas as pd
import numpy as np
from DS_Planck_Unet import val_pix, east_val, west_val
from DS_models_stat import stat_orig_cats, stat_split_cats, do_all_stats, stat_cat, change_df
import seaborn as sns
from matplotlib import cm
from matplotlib.ticker import LinearLocator

In [2]:
files3 = { 
        'pz_rot_28' : '/home/rt2122/Data/detected_cats/full_pz_rot28_thr0.1_step8.csv',
        #'pz_act_found2_22' : '/home/rt2122/Data/detected_cats/full_pz_act_found2_22_thr0.1_step8.csv',
        'pz_all_found34' : '/home/rt2122/Data/detected_cats/full_pz_all_found34_thr0.1_step8.csv',
        'br_cat' : '/home/rt2122/Data/SRGz/nilc_src_0.3deg_mar12_matched.csv'
        }

In [2]:
files_full = {'pz14' : '/home/rt2122/Data/detected_cats/full_pz14_thr0.1_step8.csv', 
        #'pz40' : '/home/rt2122/Data/detected_cats/full_pz40_thr0.1_step8.csv',
        'pz_rot_28' : '/home/rt2122/Data/detected_cats/full_pz_rot28_thr0.1_step8.csv',
        #'pz_act10' : '/home/rt2122/Data/detected_cats/full_pz_act10_thr0.1_step8.csv',
        'pz_act14' : '/home/rt2122/Data/detected_cats/full_pz_act20_thr0.1_step8.csv',
        #'pz_act_rot_drop0.1_ep9' : 
        #      '/home/rt2122/Data/detected_cats/full_pz_act_rot_drop0.1_ep9_thr0.1_step8.csv',
        'pz_act_jan_rot_drop0.1_ep6' : 
              '/home/rt2122/Data/detected_cats/full_pz_act_jan_rot_drop0.1_ep6_thr0.1_step8.csv',
        #'pz_act_feb_rot_drop0.1_ep5' : 
        #     '/home/rt2122/Data/detected_cats/full_pz_act_feb_rot_drop0.1_ep5_thr0.1_step8.csv',
        'pz_act_feb_rot_drop0.2_ep10' :
              '/home/rt2122/Data/detected_cats/full_pz_act_feb_rot_drop0.2_ep10_thr0.1_step8.csv',
        #'pz_act_feb_rot_drop0.3_ep14' :
        #     '/home/rt2122/Data/detected_cats/full_pz_act_feb_rot_drop0.3_ep14_thr0.1_step8.csv',
        'pz_act_found21':'/home/rt2122/Data/detected_cats/full_pz_act_found21_thr0.1_step8.csv',
        'pz_act_found2_22' : '/home/rt2122/Data/detected_cats/full_pz_act_found2_22_thr0.1_step8.csv',
        'pz_all_found34' : '/home/rt2122/Data/detected_cats/full_pz_all_found34_thr0.1_step8.csv'}

In [7]:
files_val= {'pz14' : '/home/rt2122/Data/detected_cats/full_pz14_thr0.1_step8.csv', 
        'pz40' : '/home/rt2122/Data/detected_cats/full_pz40_thr0.1_step8.csv',
            
        #'pz_rot19' : '/home/rt2122/Data/detected_cats/val_pz_rot19_thr0.1_step8.csv',
        'pz_rot28' : '/home/rt2122/Data/detected_cats/val_pz_rot28_thr0.1_step8.csv',
        #'pz_rot33' : '/home/rt2122/Data/detected_cats/val_pz_rot33_thr0.1_step8.csv',
            
        'pz_act10' : '/home/rt2122/Data/detected_cats/full_pz_act10_thr0.1_step8.csv',
        'pz_act14' : '/home/rt2122/Data/detected_cats/full_pz_act20_thr0.1_step8.csv',
            
        #'pz_act_rot_drop0.1_ep9' : 
        #      '/home/rt2122/Data/detected_cats/full_pz_act_rot_drop0.1_ep9_thr0.1_step8.csv',
        #'pz_act_jan_rot_drop0.1_ep6' : 
        #      '/home/rt2122/Data/detected_cats/full_pz_act_jan_rot_drop0.1_ep6_thr0.1_step8.csv',
        #'pz_act_feb_rot_drop0.1_ep5' : 
        #     '/home/rt2122/Data/detected_cats/full_pz_act_feb_rot_drop0.1_ep5_thr0.1_step8.csv',
        #'pz_act_feb_rot_drop0.2_ep10': 
        #   '/home/rt2122/Data/detected_cats/val_pz_act_feb_rot_drop0.2_ep10_thr0.1_step8.csv',
        #'pz_act_feb_rot_drop0.3_ep14' :
             #'/home/rt2122/Data/detected_cats/full_pz_act_feb_rot_drop0.3_ep14_thr0.1_step8.csv',
           #'pz_act_q_0.1_0.9_12' : '/home/rt2122/Data/detected_cats/val_pz_act_q_12_thr0.1_step8.csv',
           #'pz_act_q_0.1_0.9_14' : '/home/rt2122/Data/detected_cats/val_pz_act_q_14_thr0.1_step8.csv',
           #'pz_act_found13' : '/home/rt2122/Data/detected_cats/val_pz_act_found13_thr0.1_step8.csv',
           #'pz_act_found16':'/home/rt2122/Data/detected_cats/val_pz_act_found16_thr0.1_step8.csv',
           'pz_act_found21':'/home/rt2122/Data/detected_cats/val_pz_act_found21_thr0.1_step8.csv',
           'pz_act_found2_22' : '/home/rt2122/Data/detected_cats/val_pz_act_found2_22_thr0.1_step8.csv',
           #'pz_all_found19' : '/home/rt2122/Data/detected_cats/val_pz_all_found19_thr0.1_step8.csv',
           #'pz_all_found25' : '/home/rt2122/Data/detected_cats/val_pz_all_found25_thr0.1_step8.csv',
           'pz_all_found34' : '/home/rt2122/Data/detected_cats/val_pz_all_found34_thr0.1_step8.csv',
           #'pz_all_found2_17':'/home/rt2122/Data/detected_cats/val_pz_all_found2_17_thr0.1_step8.csv',
           #'pz_all_found2_29':'/home/rt2122/Data/detected_cats/val_pz_all_found2_29_thr0.1_step8.csv',
           #'pz_all_found2_34':'/home/rt2122/Data/detected_cats/val_pz_all_found2_34_thr0.1_step8.csv',
           #'pz_found0.8_20' : '/home/rt2122/Data/detected_cats/found0.8/we_val_pz_found0.8_20_thr0.1_step8.csv',
           #'pz_found0_28' : '/home/rt2122/Data/detected_cats/found0/we_val_pz_found0_28_thr0.1_step8.csv'
           }

In [3]:
res_df, stat_df = stat_split_cats(files_val, val_pix)
res_df

,planck_z,planck_no_z,mcxcwp,actwp,fp
pz14,0.939394,0.821429,0.155340,0.130435,949
pz40,0.931818,0.785714,0.145631,0.130435,1180
pz_rot19,0.962121,0.750000,0.165049,0.145221,1439
pz_rot28,0.962121,0.785714,0.174757,0.141544,1287
pz_rot33,0.954545,0.785714,0.155340,0.154412,1379
pz_act10,0.901515,0.821429,0.097087,0.089202,624
pz_act14,0.825758,0.750000,0.058252,0.043478,557
pz_act_rot_drop0.1_ep9,0.931818,0.750000,0.155340,0.094518,605
pz_act_jan_rot_drop0.1_ep6,0.954545,0.785714,0.106796,0.093750,693
pz_act_feb_rot_drop0.1_ep5,0.969697,0.821429,0.116505,0.163603,1843


In [3]:
comp_df, recall_df = stat_orig_cats(files3, '/home/rt2122/Data/original_catalogs/csv', 
                                    big_pix=east_val + west_val, dict_cut={})

/home/rt2122/L/git/data-segmentation-2/modules/DS_models_stat.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['found'] = False
/home/rt2122/L/git/data-segmentation-2/modules/DS_models_stat.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['b'] = sc.galactic.b.degree
/home/rt2122/L/git/data-segmentation-2/modules/DS_models_stat.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [5]:
recall_df[['PSZ2', 'MCXC', 'ACT', 'Abell', 'fp', 'all']]

,PSZ2,MCXC,ACT,Abell,fp,all
pz_rot_28,0.922018,0.419492,0.225914,0.222615,1333,1745
pz_all_found34,0.940367,0.444915,0.242525,0.250883,1463,1933
br_cat,0.963303,0.419492,0.227575,0.240283,1487,1921


In [5]:
comp_df, recall_df = stat_orig_cats(files3, '/home/rt2122/Data/original_catalogs/csv', 
                                    big_pix=list(range(48)), dict_cut={}, match_dist=400/3600, no_err=True)

/home/rt2122/L/git/data-segmentation-2/modules/DS_models_stat.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['found'] = False
/home/rt2122/L/git/data-segmentation-2/modules/DS_models_stat.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['b'] = sc.galactic.b.degree
/home/rt2122/L/git/data-segmentation-2/modules/DS_models_stat.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [9]:
recall_df_split, stat_df = stat_split_cats(files3, list(range(48)))

In [15]:
recall_df['PSZ2(z)'] = recall_df_split['planck_z']
recall_df['precision'] = 1 - (recall_df['fp'] / recall_df['all'])
recall_df

,PSZ2,MCXC,ACT,Abell,fp,all,PSZ2(z),precision
pz_rot_28,0.931034,0.470453,0.241955,0.230373,20604,23331,0.974406,0.116883
pz_all_found34,0.935269,0.473322,0.262694,0.248037,20480,23352,0.975320,0.122987
br_cat,0.747126,0.395869,0.205006,0.203140,11544,13689,0.845521,0.156695


In [12]:
comp_df41, recall_df41 = stat_orig_cats(files3, '/home/rt2122/Data/original_catalogs/csv', 
                                    big_pix=[41], dict_cut={}, match_dist=400/3600, no_err=True)

/home/rt2122/L/git/data-segmentation-2/modules/DS_models_stat.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['found'] = False
/home/rt2122/L/git/data-segmentation-2/modules/DS_models_stat.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['b'] = sc.galactic.b.degree
/home/rt2122/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata

In [14]:
recall_df41['precision'] = 1 - (recall_df['fp'] / recall_df['all'])
recall_df41

,PSZ2,MCXC,ACT,Abell,fp,all,precision
pz_rot_28,0.913793,0.500000,0.268293,0.201117,246,364,0.116883
pz_all_found34,0.913793,0.514706,0.292683,0.217877,300,430,0.122987
br_cat,0.948276,0.500000,0.283537,0.229050,400,530,0.156695


In [4]:
recall = stat_cat(files3)
recall = change_df(recall, format_s=lambda x: '{:.3f}'.format(x) if not float(x).is_integer() else x) 
print(recall.to_latex())

/home/rt2122/L/git/data-segmentation-2/modules/DS_models_stat.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  det_cat['found'] = False
/home/rt2122/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


\begin{tabular}{llllllr}
\toprule
{} &   PSZ2 &   MCXC &    ACT &  Abell & precision &    all \\
\midrule
pz\_rot\_28      &  0.917 &  0.425 &  0.219 &  0.196 &     0.108 &  23331 \\
pz\_all\_found34 &  0.923 &  0.438 &  0.236 &  0.209 &     0.113 &  23352 \\
br\_cat         &  0.741 &  0.378 &  0.190 &  0.178 &     0.148 &  13689 \\
\bottomrule
\end{tabular}



In [5]:
recall

,PSZ2,MCXC,ACT,Abell,precision,all
pz_rot_28,0.917,0.425,0.219,0.196,0.108,23331
pz_all_found34,0.923,0.438,0.236,0.209,0.113,23352
br_cat,0.741,0.378,0.190,0.178,0.148,13689


In [13]:
recall65 = stat_cat(files3, dict_cut={'max_pred' : [0.65, 2]})
recall65 = change_df(recall65, format_s=lambda x: '{:.3f}'.format(x) if not float(x).is_integer() else x)
recall65

/home/rt2122/L/git/data-segmentation-2/modules/DS_models_stat.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  det_cat['found'] = False
/home/rt2122/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,PSZ2,MCXC,ACT,Abell,precision,all
pz_rot_28,0.799,0.346,0.125,0.136,0.374,4475
pz_all_found34,0.759,0.339,0.119,0.128,0.562,2749
br_cat,0.741,0.378,0.190,0.178,0.148,13689


In [12]:
recall85 = stat_cat(files3, dict_cut={'max_pred' : [0.85, 2]})
recall85 = change_df(recall85, format_s=lambda x: '{:.3f}'.format(x) if not float(x).is_integer() else x) 
recall85

/home/rt2122/L/git/data-segmentation-2/modules/DS_models_stat.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  det_cat['found'] = False
/home/rt2122/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,PSZ2,MCXC,ACT,Abell,precision,all
pz_rot_28,0.682,0.302,0.098,0.110,0.586,2220
pz_all_found34,0.567,0.271,0.074,0.092,0.851,1176
br_cat,0.741,0.378,0.190,0.178,0.148,13689


In [14]:
recall65.index += ' 0.65'

,PSZ2,MCXC,ACT,Abell,precision,all
pz_rot_28 0.65,0.799,0.346,0.125,0.136,0.374,4475
pz_all_found34 0.65,0.759,0.339,0.119,0.128,0.562,2749
br_cat 0.65,0.741,0.378,0.190,0.178,0.148,13689


In [15]:
recall85.index += ' 0.85'

In [26]:
dict(recall65.loc['pz_rot_28 0.65'])

{'PSZ2': '0.799',
 'MCXC': '0.346',
 'ACT': '0.125',
 'Abell': '0.136',
 'precision': '0.374',
 'all': 4475}

In [28]:
res = pd.concat([pd.DataFrame(dict(recall65.loc['pz_all_found34 0.65']), index=['pz_all_found34 0.65']), 
                 pd.DataFrame(dict(recall85.loc['pz_rot_28 0.85']), index=['pz_rot_28 0.85']), 
                 pd.DataFrame(dict(recall65.loc['br_cat 0.65']), index=['br_cat 0.65'])])
print(res.to_latex())

\begin{tabular}{llllllr}
\toprule
{} &   PSZ2 &   MCXC &    ACT &  Abell & precision &    all \\
\midrule
pz\_all\_found34 0.65 &  0.759 &  0.339 &  0.119 &  0.128 &     0.562 &   2749 \\
pz\_rot\_28 0.85      &  0.682 &  0.302 &  0.098 &  0.110 &     0.586 &   2220 \\
br\_cat 0.65         &  0.741 &  0.378 &  0.190 &  0.178 &     0.148 &  13689 \\
\bottomrule
\end{tabular}

